In [ ]:
!pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import SpatialDropout1D
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional
from keras.layers import Dropout


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,auc
from tensorflow.keras.layers import Dense, LSTM,SimpleRNN, Activation, Input
from keras.layers import CuDNNLSTM
from keras_preprocessing.sequence import pad_sequences
from sklearn.metrics import roc_auc_score,precision_score, recall_score, f1_score,accuracy_score

In [ ]:
def Evaluation (Method,Comment,Actual, Predicted,Scores):
    ROC_AUC=roc_auc_score(Actual,Predicted)
    Precision=precision_score(Actual,Predicted)
    Accuracy=accuracy_score(Actual,Predicted)
    Recall=recall_score(Actual,Predicted)
    F1=f1_score(Actual,Predicted)
    print('Accuracy=',Accuracy)
    Scores[Comment] = {'Method':Method,'ROC_AUC':ROC_AUC,'Accuracy':Accuracy,
                       'Precision':Precision,'Recall':Recall,'F1':F1}

In [ ]:
#return dictionary with word: embedding_vector
def load_embedding(fname):
    embeddings_index = {}
    f = open(fname, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        try:
            embedding_vector = np.asarray(values[1:], dtype='float32')
        except:
            pass
        embeddings_index[word] = embedding_vector
    f.close()

    return embeddings_index

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df  = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisStockProject/dataset/pre_process_news_days.csv', encoding = "ISO-8859-1")

In [ ]:
from ast import literal_eval
df['news_stemmed'] = df['news_stemmed'].apply(lambda x: literal_eval(x))

Split Data

In [ ]:
# Split the data into train, validation, and test sets
train_data = df[(df['Date'] >= '2008-08-08') & (df['Date'] <= '2013-12-31')]
val_data = df[(df['Date'] >= '2014-01-02') & (df['Date'] <= '2014-12-31')]
test_data = df[(df['Date'] >= '2015-01-02') & (df['Date'] <= '2016-07-01')]

In [ ]:
text_train = train_data.news_stemmed.values
text_val = val_data.news_stemmed.values
text_test = test_data.news_stemmed.values

In [ ]:
EMBEDDING_DIM = 50
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_train)
word_index = tokenizer.word_index
MAX_WORDS = len(word_index)+1
print('number of unique tokens are: ', len(word_index))

number of unique tokens are:  23270


In [ ]:
max_sequence=200

embeddings_index = load_embedding("/content/drive/MyDrive/Colab Notebooks/SentimentAnalysisStockProject/dataset/glove.6B.50d.txt")

embedding_matrix = np.random.random((MAX_WORDS, EMBEDDING_DIM))

for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [ ]:
train_sequences = tokenizer.texts_to_sequences(text_train)
train_bow_data = tokenizer.sequences_to_matrix(train_sequences, mode='binary')
X_train = pad_sequences(train_bow_data, maxlen=max_sequence,padding='post')

val_sequences = tokenizer.texts_to_sequences(text_val)
val_bow_data = tokenizer.sequences_to_matrix(val_sequences, mode='binary')
X_val = pad_sequences(val_bow_data, maxlen=max_sequence,padding='post')

test_sequences = tokenizer.texts_to_sequences(text_test)
test_bow_data = tokenizer.sequences_to_matrix(test_sequences, mode='binary')
X_test = pad_sequences(test_bow_data, maxlen=max_sequence,padding='post')

In [ ]:
sc={}

for i, lbl in enumerate(['Label','Label_1day','Label_2day', 'Label_3day', 'Label_4day', 'Label_5day']):
  Y_train = np.array(train_data[lbl])
  Y_val = np.array(val_data[lbl])
  Y_test = np.array(test_data[lbl])

  hidden_dims = 20
  np.random.seed(44)
  model_LSTM = Sequential()
  model_LSTM.add(Embedding(input_dim=MAX_WORDS, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_sequence, trainable=True))
  model_LSTM.add(CuDNNLSTM(hidden_dims))
  model_LSTM.add(Dropout(0.2))
  model_LSTM.add(Dense(units=1, activation='sigmoid'))
  model_LSTM.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)
  hist_LSTM = model_LSTM.fit(X_train, Y_train,
                            validation_data=(X_val, Y_val),
                            epochs=10, batch_size=32, callbacks=[es])
  y_prob = model_LSTM.predict(X_test)
  y_pred_LSTM = y_prob.round()
  Evaluation ('BoW + LSTM',lbl,Y_test, y_pred_LSTM,sc)

Epoch 1/10
43/43 [==============================] - 4s 57ms/step - loss: 0.6980 - accuracy: 0.5121 - val_loss: 0.6881 - val_accuracy: 0.5516
Epoch 2/10
43/43 [==============================] - 1s 34ms/step - loss: 0.6945 - accuracy: 0.5166 - val_loss: 0.6886 - val_accuracy: 0.5516
Epoch 3/10
43/43 [==============================] - 2s 38ms/step - loss: 0.6922 - accuracy: 0.5247 - val_loss: 0.6884 - val_accuracy: 0.5516
Epoch 4/10
43/43 [==============================] - 2s 39ms/step - loss: 0.6952 - accuracy: 0.5195 - val_loss: 0.6878 - val_accuracy: 0.5516
Epoch 5/10
43/43 [==============================] - 2s 35ms/step - loss: 0.6930 - accuracy: 0.5232 - val_loss: 0.6879 - val_accuracy: 0.5516
Epoch 6/10
43/43 [==============================] - 1s 34ms/step - loss: 0.6918 - accuracy: 0.5291 - val_loss: 0.6894 - val_accuracy: 0.5516
Epoch 7/10
43/43 [==============================] - 2s 40ms/step - loss: 0.6904 - accuracy: 0.5350 - val_loss: 0.6879 - val_accuracy: 0.5516
Epoch 8/10
12

In [ ]:
df_sc_LSTM = pd.DataFrame.from_dict(sc, orient='index')
df_sc_LSTM.sort_values(by=['ROC_AUC','Accuracy'],ascending=False,inplace=True)
df_sc_LSTM

,Method,ROC_AUC,Accuracy,Precision,Recall,F1
Label_2day,BoW + LSTM,0.5,0.547619,0.000000,0.0,0.000000
Label_4day,BoW + LSTM,0.5,0.534392,0.000000,0.0,0.000000
Label_3day,BoW + LSTM,0.5,0.531746,0.000000,0.0,0.000000
Label_5day,BoW + LSTM,0.5,0.531746,0.000000,0.0,0.000000
Label,BoW + LSTM,0.5,0.507937,0.507937,1.0,0.673684
Label_1day,BoW + LSTM,0.5,0.507937,0.000000,0.0,0.000000
